In [2]:
from train_nep.config import get_nep_config
from train_nep.dataset import StructureDataset, collate_fn
from train_nep.model import NEP
from train_nep.alchemy import Alchemy
from torch.utils.data import DataLoader

config = get_nep_config()

train_dataset = StructureDataset(filepath="../Repository/Training_NEP/PbTe.xyz", para=config)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

model = NEP(config)

alchemy = Alchemy(
    model=model,
    training_set=train_loader,
    save_path="nep_model.pt"
)

alchemy.fit(epochs=100)
alchemy.load("nep_model.pt")

开始训练，总共 100 轮
设备: cpu
模型参数数量: 1773
--------------------------------------------------
  [Batch 1] Loss = 17.094135 | Energy = 16.897198 | Forces = 0.196938 | Virial = 0.000000
  [Batch 2] Loss = 16.519854 | Energy = 16.314323 | Forces = 0.205531 | Virial = 0.000000
  [Batch 3] Loss = 16.039165 | Energy = 15.868309 | Forces = 0.170856 | Virial = 0.000000
  [Batch 4] Loss = 15.314266 | Energy = 15.052928 | Forces = 0.261338 | Virial = 0.000000
Epoch   1/100: Train Loss = 16.241855 | Energy = 16.033190 | Forces = 0.208666 | Virial = 0.000000
  [Batch 1] Loss = 14.894033 | Energy = 14.677477 | Forces = 0.216557 | Virial = 0.000000
  [Batch 2] Loss = 14.440402 | Energy = 14.235116 | Forces = 0.205286 | Virial = 0.000000
  [Batch 3] Loss = 13.992927 | Energy = 13.811991 | Forces = 0.180936 | Virial = 0.000000
  [Batch 4] Loss = 13.335196 | Energy = 13.063835 | Forces = 0.271362 | Virial = 0.000000
Epoch   2/100: Train Loss = 14.165640 | Energy = 13.947105 | Forces = 0.218535 | Virial = 0.000

Loss/train_energy,█▇▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_forces,▂▃▇▇█▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_total,█▇▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss/train_virial,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇███
Loss/train_energy,0.00128
Loss/train_forces,0.10166
Loss/train_total,0.10294
Loss/train_virial,0
epoch,100



训练完成!
最佳验证损失: inf
✓ 已加载检查点: nep_model.pt


In [6]:
import torch
from wizard.io import read_xyz
from train_nep.interface import NEPCalculator
from train_nep.config import get_nep_config

test_atoms = read_xyz("../Repository/Training_NEP/PbTe.xyz")[0]  

para = get_nep_config()  
nep_calc = NEPCalculator(model_path="nep_model.pt", para=para) 

test_atoms.set_calculator(nep_calc)

energy = test_atoms.get_potential_energy()
forces = test_atoms.get_forces()

print(f"Structure: {len(test_atoms)} atoms")
print(f"Chemical formula: {test_atoms.get_chemical_formula()}")
print(f"Predicted energy: {energy:.6f} eV")
print(f"Energy per atom: {energy/len(test_atoms):.6f} eV/atom")
print(f"Forces shape: {forces.shape}")

if 'energy' in test_atoms.info:
    true_energy = test_atoms.info['energy']
    print(f"\nComparison with true values:")
    print(f"True energy per atom: {true_energy:.6f} eV")
    print(f"Energy error: {(abs(energy - true_energy)/len(test_atoms)):.6f} eV/atom")

if 'forces' in test_atoms.info:
    true_forces = test_atoms.info['forces']
    force_errors = torch.norm(torch.tensor(forces - true_forces), dim=1)
    print(f"Mean force error: {force_errors.mean():.6f} eV/Å")

Structure: 250 atoms
Chemical formula: Pb125Te125
Predicted energy: -930.870544 eV
Energy per atom: -3.723482 eV/atom
Forces shape: (250, 3)

Comparison with true values:
True energy per atom: -937.191000 eV
Energy error: 0.025282 eV/atom
Mean force error: 0.470272 eV/Å


/var/folders/pt/1s6hzgb542q02c62v8qffkyc0000gn/T/ipykernel_75917/4271065278.py:11: FutureWarning: Please use atoms.calc = calc
  test_atoms.set_calculator(nep_calc)
